# The Battle of the Neighborhoods

## Introduction/Business Problem

*Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.*

Our hypothetical client is a restauranteer who is interested in opening up a gastropub in Manchester, UK. Because of how niche a gastropub is the client wants to find an area with a relatively low amount of gastropubs so that he is not too close to any competitors. We have taken on the role of advisor in this situation, and want to advise our client on which neighborhood is best for his gastropub. With our information the client will easily be able to search for vacant properties which he can turn into a gastropub.

## Data

*Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.*

We will use the following data to best advise our client:
- A Wikipedia page with all neighborhoods in Manchester
- The latitude and longitude of said neighborhoods, optained using the geolocator (we are using ArcGIS at is is free to use and gives accurate results)
- Data concerning venues for each neighborhoods obtained using the Foursquare API

---

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## Scraping the webpage

In [2]:
# send a GET request for the webpage
data = requests.get("https://en.wikipedia.org/wiki/Category:Areas_of_Manchester").text # parse data from the html into a beautifulsoup object

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [3]:
# create an empty list to store neighborhood data
neighs = []

# append the data into the list while removing all wards (wards overlap with neighborhoods already existing in the list)
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    
    text = row.text
    
    if "ward" not in text:
        neighs.append(text)

In [4]:
# create a new DataFrame from the list
df = pd.DataFrame({"Neighborhood": neighs})

df.head()

,Neighborhood
0,Abbey Hey
1,Alport Town
2,Ancoats
3,Ardwick
4,Ardwick Green


In [5]:
df.shape

(64, 1)

## Getting the geographical coordinates

In [6]:
# define a function to get coordinates
def get_latlng(neighborhood):
    g = geocoder.arcgis('{}, Manchester, UK'.format(neighborhood))
    lat_lng_coords = g.latlng
    return lat_lng_coords

In [7]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [8]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [9]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [10]:
# check the neighborhoods and the coordinates
print(df.shape)
df

(64, 3)


,Neighborhood,Latitude,Longitude
0,Abbey Hey,53.469912,-2.158939
1,Alport Town,53.449186,-2.270495
2,Ancoats,53.481175,-2.227869
3,Ardwick,53.471823,-2.220136
4,Ardwick Green,53.471936,-2.224624
5,Baguley,53.399432,-2.294060
6,Barlow Moor,53.426562,-2.266241
7,"Belle Vue, Manchester",53.463060,-2.197180
8,Benchill,53.389117,-2.265841
9,"Beswick, Manchester",53.481160,-2.197640


## Creating a map of Manchester with neighborhoods superimposed on top

In [11]:
# get the coordinates of Kuala Lumpur
address = 'Manchester, UK'

geolocator = Nominatim(user_agent="manchester_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manchester, England {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manchester, England 53.4794892, -2.2451148.


In [12]:
# create map of Toronto using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
    
map

In [13]:
# save the map as HTML file
map.save('manchester.html')

## Using the Foursquare API to explore the neighborhoods

In [37]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20210527' # Foursquare API version

**Now, let's get the top 100 venues for each neighborhood in a radius of 2000 meters.**

In [15]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [16]:
# convert the venues list into a new dataframe
venues_df = pd.DataFrame(venues)

# define column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4315, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abbey Hey,53.469912,-2.158939,The Gym Manchester Ashton Old Road,53.473077,-2.164137,Gym / Fitness Center
1,Abbey Hey,53.469912,-2.158939,Costa Coffee,53.473335,-2.164748,Coffee Shop
2,Abbey Hey,53.469912,-2.158939,Lidl,53.472882,-2.165574,Supermarket
3,Abbey Hey,53.469912,-2.158939,Debdale Park,53.458288,-2.160558,Park
4,Abbey Hey,53.469912,-2.158939,Wright Robinson College,53.466332,-2.150999,General College & University


**Let's check how many venues were returned for each neighorhood**

In [17]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Abbey Hey,32,32,32,32,32,32
Alport Town,85,85,85,85,85,85
Ancoats,100,100,100,100,100,100
Ardwick,100,100,100,100,100,100
Ardwick Green,100,100,100,100,100,100
Baguley,35,35,35,35,35,35
Barlow Moor,92,92,92,92,92,92
"Belle Vue, Manchester",28,28,28,28,28,28
Benchill,33,33,33,33,33,33


**Let's find out how many unique categories can be curated from all the returned venues**

In [18]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 192 uniques categories.


In [19]:
# check if the results contain the word "Gastropub"
"Gastropub" in venues_df['VenueCategory'].unique()

True

## Analyzing Each Neighborhood

In [20]:
# one hot encoding
df_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

print(df_onehot.shape)
df_onehot.head()

(4315, 193)


,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Betting Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Caribbean Restaurant,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Disc Golf,Discount Store,Donut Shop,Duty-free Shop,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Newsagent,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rugby Pitch,Sandwich Place,Science Museum,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Track Stadium,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Yemeni Restaurant
0,Abbey Hey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abbey Hey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abbey Hey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abbey Hey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [21]:
venue_grouped = df_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(venue_grouped.shape)
venue_grouped

(64, 193)


,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Betting Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Caribbean Restaurant,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Disc Golf,Discount Store,Donut Shop,Duty-free Shop,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Newsagent,Outdoor Sculpture,Outdoor Supply Store,Park,Pastry Shop,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rugby Pitch,Sandwich Place,Science Museum,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Track Stadium,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Yemeni Restaurant
0,Abbey Hey,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.031250,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.031250,0.000000,0.000000,0.031250,0.000000,0.093750,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.00,0.000000,0.000000,0.093750,0.031

In [22]:
# print neighborhoods with a gastropub frequency higher than 0
len(venue_grouped[venue_grouped["Gastropub"] > 0])

14

**Let's create a new dataframe for gastropubs**

In [23]:
df_gastro = venue_grouped[["Neighborhoods","Gastropub"]]

In [24]:
df_gastro.head()

,Neighborhoods,Gastropub
0,Abbey Hey,0.000000
1,Alport Town,0.011765
2,Ancoats,0.000000
3,Ardwick,0.000000
4,Ardwick Green,0.000000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [25]:
# set number of clusters
kclusters = 3

kl_clustering = df_gastro.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
df_merged = df_gastro.copy()

# add clustering labels
df_merged["Cluster Labels"] = kmeans.labels_

In [27]:
df_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
df_merged.head()

,Neighborhood,Gastropub,Cluster Labels
0,Abbey Hey,0.000000,1
1,Alport Town,0.011765,2
2,Ancoats,0.000000,1
3,Ardwick,0.000000,1
4,Ardwick Green,0.000000,1


In [28]:
# adding latlong for each neighborhood
df_merged = df_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(df_merged.shape)
df_merged.head() # check the last columns!

(64, 5)


,Neighborhood,Gastropub,Cluster Labels,Latitude,Longitude
0,Abbey Hey,0.000000,1,53.469912,-2.158939
1,Alport Town,0.011765,2,53.449186,-2.270495
2,Ancoats,0.000000,1,53.481175,-2.227869
3,Ardwick,0.000000,1,53.471823,-2.220136
4,Ardwick Green,0.000000,1,53.471936,-2.224624


In [29]:
# sort the results by Cluster Labels
print(df_merged.shape)
df_merged.sort_values(["Cluster Labels"], inplace=True)
df_merged

(64, 5)


,Neighborhood,Gastropub,Cluster Labels,Latitude,Longitude
18,Chorltonville,0.029412,0,53.437550,-2.274520
16,Chorlton-cum-Hardy,0.029851,0,53.437936,-2.269066
45,Northenden,0.032258,0,53.407111,-2.258162
0,Abbey Hey,0.000000,1,53.469912,-2.158939
33,Ladybarn,0.000000,1,53.438706,-2.213825
34,Levenshulme,0.000000,1,53.441961,-2.185025
35,Longsight,0.000000,1,53.457224,-2.206326
36,Manchester city centre,0.000000,1,53.486200,-2.234090
37,Merseybank,0.000000,1,53.422775,-2.262769
38,Miles Platting,0.000000,1,53.491134,-2.213318


**Finally, let's visualize the resulting clusters**

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='Stamen Toner')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    labelstring = ''
    if cluster == 0:
        labelstring = "High"
    elif cluster == 1:
        labelstring = "Low"
    else:
        labelstring = "Medium"
    label = folium.Popup(str(poi) + " " + labelstring, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examining the clusters

#### Cluster 0 - High gastropubs

In [32]:
df_merged.loc[df_merged['Cluster Labels'] == 0]

,Neighborhood,Gastropub,Cluster Labels,Latitude,Longitude
18,Chorltonville,0.029412,0,53.437550,-2.274520
16,Chorlton-cum-Hardy,0.029851,0,53.437936,-2.269066
45,Northenden,0.032258,0,53.407111,-2.258162


#### Cluster 1 - Low gastropubs

In [33]:
df_merged.loc[df_merged['Cluster Labels'] == 1]

,Neighborhood,Gastropub,Cluster Labels,Latitude,Longitude
0,Abbey Hey,0.0,1,53.469912,-2.158939
33,Ladybarn,0.0,1,53.438706,-2.213825
34,Levenshulme,0.0,1,53.441961,-2.185025
35,Longsight,0.0,1,53.457224,-2.206326
36,Manchester city centre,0.0,1,53.486200,-2.234090
37,Merseybank,0.0,1,53.422775,-2.262769
38,Miles Platting,0.0,1,53.491134,-2.213318
40,"Moston, Manchester",0.0,1,53.513367,-2.207417
41,New Islington,0.0,1,53.484295,-2.222530
42,New Moston,0.0,1,53.516688,-2.168234


### Cluster 2 - Medium gastropubs

In [34]:
df_merged.loc[df_merged['Cluster Labels'] == 2]

,Neighborhood,Gastropub,Cluster Labels,Latitude,Longitude
1,Alport Town,0.011765,2,53.449186,-2.270495
60,"Whalley Range, Manchester",0.014085,2,53.453600,-2.263870
32,Hulme,0.010000,2,53.467052,-2.246985
14,Castlefield,0.010000,2,53.476060,-2.252190
55,Spinningfields,0.010000,2,53.480193,-2.249159
23,Didsbury,0.010000,2,53.417495,-2.230388
50,Parrs Wood,0.010000,2,53.416380,-2.220310
39,Moss Side,0.020000,2,53.459014,-2.244017
29,"Greenheys, Manchester",0.010000,2,53.462187,-2.241189
56,"St John's, Manchester",0.010000,2,53.477678,-2.251116


## Conclusion

Seeing the distribution of gastropubs in Manchester, our client would do well to avoid any neighborhood in Cluster 0 as there are already a lot of gastropubs here. These only appear in southern Manchester.

While neighborhoods with a medium amount of gastropubs are a decent choice, no gastropubs might be preferable.

Interestingly enough, there seems to be almost no gastropubs in east Manchester. As such, we would advise our client to consider opening a gastropub there.